# CMPSC 441: Homework 3

In [ ]:
student_name = 'Aman Sahu'
student_email = 'ajs9219@psu.edu'

## Imports

In [1]:
from hw3_utils import *
from collections import deque
import math

## 1. Best-First, Uniform-Cost, A-Star Search Algorithms

In [2]:
def best_first_search(problem):
    node = Node(problem.init_state, heuristic=problem.h(problem.init_state))
    frontier = deque([node])         # queue: popleft/append-sorted
    explored = [problem.init_state]  # used as "visited"

    def getNodeHeuristic(n):
        return n.heuristic

    while frontier:
        frontier = deque(sorted(frontier, key=getNodeHeuristic)) # lowest heuristic at front
        node = frontier.popleft()

        # goal test
        if problem.goal_test(node.state):
            return node

        explored.append(node.state)

        for item in node.expand(problem):
            if item.state not in explored and all(item.state != n.state for n in frontier): # all these conds are true
                frontier.append(item)
                
            elif any(item.state == n.state and item.heuristic < n.heuristic for n in frontier): # only one needs to be true
                frontier = deque([item if n.state == item.state else n for n in frontier]) # adding lower heuristic child

    return Node(None)

In [3]:
def uniform_cost_search(problem):
    node = Node(problem.init_state)
    frontier = deque([node])         # queue: popleft/append-sorted
    explored = []                    # used as "expanded" (not "visited")

    def getNodePathCost(n):
        return n.path_cost
    
    while frontier:
        frontier = deque(sorted(frontier, key=getNodePathCost)) # lowest heuristic at front
        node = frontier.popleft()

        # goal test
        if problem.goal_test(node.state):
            return node

        explored.append(node.state)

        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)

            elif child in frontier:
                for existing_node in frontier: # could use next()?
                    if existing_node.state == child.state:
                        if child.path_cost < existing_node.path_cost:
                            frontier.remove(existing_node)
                            frontier.append(child) # replacing with lower cost child 
                        break

    return Node(None)

In [13]:
def a_star_search(problem):
    node = Node(problem.init_state, heuristic=problem.h(problem.init_state))
    frontier = deque([node])         # queue: popleft/append-sorted
    explored = []                    # used as "expanded" (not "visited")
    
    def getNodeTotalCost(n):
        return n.path_cost + n.heuristic

    while frontier:
        frontier = deque(sorted(frontier, key=getNodeTotalCost))
        node = frontier.popleft()

        # goal test
        if problem.goal_test(node.state):
            return node

        explored.append(node.state)

        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)

            elif child in frontier:
                for existing_node in frontier:
                    if existing_node.state == child.state:
                        if getNodeTotalCost(child) < getNodeTotalCost(existing_node):
                            frontier.remove(existing_node)
                            frontier.append(child) # replacing with lower cost child
                        break

    return Node(None)


## 2. N-Queens Problem

In [5]:
class NQueensProblem(Problem):
    """
    The implementation of the class NQueensProblem related
    to Homework 2 is given for those students who were not
    able to complete it in Homework 2.
    
    Note that you do not have to use this implementation.
    Instead, you can use your own implementation from
    Homework 2.

    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    """
    
    def __init__(self, n):
        super().__init__(tuple([-1] * n))
        self.n = n
        

    def actions(self, state):
        if state[-1] != -1:   # if all columns are filled
            return []         # then no valid actions exist
        
        valid_actions = list(range(self.n))
        col = state.index(-1) # index of leftmost unfilled column
        for row in range(self.n):
            for c, r in enumerate(state[:col]):
                if self.conflict(row, col, r, c) and row in valid_actions:
                    valid_actions.remove(row)
                    
        return valid_actions

        
    def result(self, state, action):
        col = state.index(-1) # leftmost empty column
        new = list(state[:])  
        new[col] = action     # queen's location on that column
        return tuple(new)

    
    def goal_test(self, state):
        if state[-1] == -1:   # if there is an empty column
            return False;     # then, state is not a goal state

        for c1, r1 in enumerate(state):
            for c2, r2 in enumerate(state):
                if (r1, c1) != (r2, c2) and self.conflict(r1, c1, r2, c2):
                    return False
        return True

    
    def conflict(self, row1, col1, row2, col2):
        return row1 == row2 or col1 == col2 or abs(row1-row2) == abs(col1-col2)

        
    def g(self, cost, from_state, action, to_state):
        """
        Return path cost from start state to to_state via from_state.
        The path from start_state to from_state costs the given cost
        and the action that leads from from_state to to_state
        costs 1.
        """
        return cost + 1



    def h(self, state):
        """
        Returns the heuristic value for the given state.
        Use the total number of conflicts in the given
        state as a heuristic value for the state.
        """
        
        conflicts = 0
        n = len(state)
        
        # checking all pairs
        for c1 in range(0,n):
            r1 = state[c1]
            for c2 in range(c1 + 1, n):
                r2 = state[c2]
                
                if self.conflict(r1, c1, r2, c2):
                    conflicts += 1

        return 2 * conflicts # doubling for simplicity

## 3. Graph Problem

In [7]:
class GraphProblem(Problem):
    """
    The implementation of the class GraphProblem related
    to Homework 2 is given for those students who were
    not able to complete it in Homework 2.
    
    Note that you do not have to use this implementation.
    Instead, you can use your own implementation from
    Homework 2.

    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    >>>> USE THIS IMPLEMENTATION AT YOUR OWN RISK <<<<
    """
    
    
    def __init__(self, init_state, goal_state, graph):
        super().__init__(init_state, goal_state)
        self.graph = graph

        
    def actions(self, state):
        """Returns the list of adjacent states from the given state."""
        return list(self.graph.get(state).keys())

    
    def result(self, state, action):
        """Returns the resulting state by taking the given action.
            (action is the adjacent state to move to from the given state)"""
        return action

    
    def goal_test(self, state):
        return state == self.goal_state

    
    def g(self, cost, from_state, action, to_state):
        """
        Returns the path cost from root to to_state.
        Note that the path cost from the root to from_state
        is the give cost and the given action taken at from_state
        will lead you to to_state with the cost associated with
        the action.
        """
        if to_state in self.actions(from_state) and action == to_state:
            return cost + self.graph.get(from_state)[to_state]
        return 0
    

    def h(self, state):
        """
        Returns the heuristic value for the given state. Heuristic
        value of the state is calculated as follows:
        1. if an attribute called 'heuristics' exists:
           - heuristics must be a dictionary of states as keys
             and corresponding heuristic values as values
           - so, return the heuristic value for the given state
        2. else if an attribute called 'locations' exists:
           - locations must be a dictionary of states as keys
             and corresponding GPS coordinates (x, y) as values
           - so, calculate and return the straight-line distance
             (or Euclidean norm) from the given state to the goal
             state
        3. else
           - cannot find nor calculate heuristic value for given state
           - so, just return a large value (i.e., infinity)
        """
        if hasattr(self.graph, 'heuristics'): # preferring heuristic
            return self.graph.heuristics.get(state, math.inf)

        elif hasattr(self.graph, 'locations'):
            if self.goal_state in self.graph.locations and state in self.graph.locations:
                x1, y1 = self.graph.locations[state][0], self.graph.locations[state][1] # from state
                x2, y2 = self.graph.locations[self.goal_state][0], self.graph.locations[self.goal_state][1] # to state

                # euclidean normß
                return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        else:
            return math.inf



## 4. Eight Puzzle

In [9]:
class EightPuzzle(Problem):
    def __init__(self, init_state, goal_state=(1,2,3,4,5,6,7,8,0)):
        self.init_state = init_state
        self.goal_state = goal_state
        self.zero_cell = self.init_state.index(0)
        self.side = math.sqrt(len(init_state))
        

    def actions(self, state):
        res = []
        
        curr_zero_cell = state.index(0) # idx of 0
        row, col = curr_zero_cell // self.side, curr_zero_cell % self.side # getting row value dividing by side length, getting col value by getting remainder using side length

        if row > 0: # not first row
            res.append('UP')
            
        if row < self.side - 1: # not the last row
            res.append('DOWN')
            
        if col > 0: # not the first col
            res.append('LEFT')
            
        if col < self.side - 1: # not the last col
            res.append('RIGHT')

        return res
            
            

    
    def result(self, state, action):
        if action not in self.actions(state):
            return state

        state = list(state)
        curr_zero_cell_idx = state.index(0)

        if action == 'UP':
            to_cell_idx = curr_zero_cell_idx - 3

        elif action == 'DOWN':
            to_cell_idx = curr_zero_cell_idx + 3

        elif action == 'LEFT':
            to_cell_idx = curr_zero_cell_idx - 1

        elif action == 'RIGHT':
            to_cell_idx = curr_zero_cell_idx + 1

        # swapping the elements
        state[to_cell_idx], state[curr_zero_cell_idx] = state[curr_zero_cell_idx], state[to_cell_idx]
            

        return tuple(state)
        

    
    def goal_test(self, state):
        return state == self.goal_state
    

    def g(self, cost, from_state, action, to_state):
        """
        Return path cost from root to to_state via from_state.
        The path from root to from_state costs the given cost
        and the action that leads from from_state to to_state
        costs 1.
        """
        return cost + 1
    

    def h(self, state):
        """
        Returns the heuristic value for the given state.
        Use the sum of the Manhattan distances of misplaced
        tiles to their final positions.
        """
        final_distance = 0
        for i in range(0, len(state)):
            if state[i] != 0:
                curr_item_row, curr_item_col = i // self.side,  i % self.side # getting row value dividing by side length, getting col value by getting remainder using side length

                goal_idx = self.goal_state.index(state[i])
                goal_row, goal_col = goal_idx // self.side,  goal_idx % self.side
                
                distance = abs(curr_item_row - goal_row) + abs(curr_item_col - goal_col)
                final_distance += distance

        return int(final_distance)



In [6]:
# queens = NQueensProblem(8)

# print(queens.h((-1,-1,-1,-1,-1,-1,-1,-1)))
# print(queens.h((7,-1,-1,-1,-1,-1,-1,-1)))
# print(queens.h((7,1,-1,-1,-1,-1,-1,-1)))
# print(queens.h((7,1,3,-1,-1,-1,-1,-1)))
# print(queens.h((7,1,3,0,-1,-1,-1,-1)))
# print(queens.h((7,1,3,0,6,-1,-1,-1)))
# print(queens.h((7,1,3,0,6,4,-1,-1)))
# print(queens.h((7,1,3,0,6,4,2,-1)))
# print(queens.h((7,1,3,0,6,4,2,5)))

In [8]:
# romania_map = Graph(romania_roads, False)
# romania_map.locations = romania_city_positions



# romania = GraphProblem('Arad', 'Bucharest', romania_map)

# print(romania.h('Arad'))
# print(romania.h('Sibiu'))
# print(romania.h('Fagaras'))
# print(romania.h('Pitesti'))
# print(romania.h('Rimnicu'))
# print(romania.h('Bucharest'))

# roads = dict(S = dict(A=1, B=2), A = dict(C=1),
#             B = dict(C=2), C=dict(G=100))

# roads_h = dict(S=90, A=100, B=88, C=100, G=0)
# roads_map = Graph(roads, True)
# roads_map.heuristics = roads_h

# problem = GraphProblem('S', 'G', roads_map)
# print(problem.h('S'))
# print(problem.h('A'))
# print(problem.h('B'))
# print(problem.h('C'))
# print(problem.h('G'))

In [10]:
# puzzle = EightPuzzle((1,0,3,4,2,5,7,8,6))

# print(puzzle.actions((0,1,2,3,4,5,6,7,8)))
# print(puzzle.actions((6, 3, 5, 1, 8, 4, 2, 0, 7)))
# print(puzzle.actions((4, 8, 1, 6, 0, 2, 3, 5, 7)))
# print(puzzle.actions((1, 0, 6, 8, 7, 5, 4, 2, 3)))
# print(puzzle.actions((1, 2, 3, 4, 5, 6, 7, 8, 0)))

# print(puzzle.result((0,1,2,3,4,5,6,7,8), 'DOWN'))
# print(puzzle.result((6,3,5,1,8,4,2,0,7), 'LEFT'))
# print(puzzle.result((3,4,1,7,6,0,2,8,5), 'UP'))
# print(puzzle.result((1,8,4,7,2,6,3,0,5), 'RIGHT'))

# print(puzzle.goal_state)
# print(puzzle.h((1,2,3,4,5,0,7,8,6)))
# print(puzzle.h((1,2,0,4,5,3,7,8,6)))
# print(puzzle.h((1,0,2,4,5,3,7,8,6)))
# print(puzzle.h((4,1,2,0,5,3,7,8,6)))
# print(puzzle.h((4,1,2,6,8,0,3,5,7)))

In [ ]:
# q = NQueensProblem(8)
# print(best_first_search(q).solution())
# print(uniform_cost_search(q).solution())
# print(a_star_search(q).solution())

# romania_map = Graph(romania_roads, False)
# romania_map.locations = romania_city_positions

# g = GraphProblem('Arad', 'Bucharest', romania_map)

# map1 = Graph(best_graph_edges, True)
# map1.heuristics = best_graph_h

# g = GraphProblem('S', 'G', map1)

# map2 = Graph(a_star_graph_edges, True)
# map2.heuristics = a_star_graph_consistent_h

# g = GraphProblem('S', 'G', map2)

# print(best_first_search(g).solution())
# print(uniform_cost_search(g).solution())
# print(a_star_search(g).solution())

# e = EightPuzzle((3,4,1,7,6,0,2,8,5))
# print(best_first_search(e).solution())
# print(a_star_search(e).solution())